In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [ ]:
loader = PyPDFLoader("book/ullman_the_complete_book.pdf")
pages = loader.load_and_split()

In [ ]:
pages = pages[:600]

In [2]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
# db = Chroma.from_documents(pages, embedding_function,persist_directory="./chroma_db")

d:\Project\RAG_for_book\rag_evn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Project\RAG_for_book\rag_evn\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
db = Chroma(persist_directory= "./chroma_db", embedding_function=embedding_function)

In [ ]:
# query it


In [33]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
import os
import textwrap
from IPython.display import Markdown
load_dotenv()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [28]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [29]:
context = docs[0].page_content

prompt = make_prompt(query, context)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'what is Modifying Relation Schemas'
  PASSAGE: '2.3. DEFINING  A RELATION SCHEMA IN  SQL 33 The gender attribute has values that are a single letter, M  or F. Thus, we  can safely use a single character as the type of this attribute. Finally, the  b irth d a te  attribute naturally deserves the data type D A TE. □ 2.3.4 Modifying Relation Schemas We now know how to declare a table. But what if we need to change the schema  of the table after it has been in use for a long time and has many tuples in its  current instance? We can remove the

In [22]:
query = "what is Modifying Relation Schemas"
docs = db.similarity_search(query)
docs

[Document(page_content='2.3. DEFINING  A RELATION SCHEMA IN  SQL 33\nThe gender attribute has values that are a single letter, M  or F. Thus, we \ncan safely use a single character as the type of this attribute. Finally, the \nb irth d a te  attribute naturally deserves the data type D A TE. □\n2.3.4 Modifying Relation Schemas\nWe now know how to declare a table. But what if we need to change the schema \nof the table after it has been in use for a long time and has many tuples in its \ncurrent instance? We can remove the entire table, including all of its current \ntuples, or we could change the schema by adding or deleting attributes.\nWe can delete a relation R by the SQL statement:\nD R O P TABLE R;\nRelation R is no longer part of the database schema, and we can no longer \naccess any of its tuples.\nMore frequently than we would drop a relation that is part of a long-lived \ndatabase, we may need to modify the schema of an existing relation. These \nmodifications are done by a st

In [39]:
docs[0].metadata

{'page': 69, 'source': 'book/ullman_the_complete_book.pdf'}

In [31]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')
answer = model.generate_content(prompt)
answer

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Modifying a relation schema, or table definition, in SQL involves making changes to the existing schema, such as adding or deleting attributes.   To add an attribute, the ALTER TABLE statement can be used with the ADD keyword, followed by the attribute name and data type.   For example, to add a phone attribute to the MovieStar table as a fixed-length string of 16 bytes, the following statement would be used: ALTER TABLE MovieStar ADD phone CHAR(16).   Similarly, to delete an attribute, the ALTER TABLE statement can be used with the DROP keyword, followed by the attribute name.   For example, to delete the birthdate attribute from the MovieStar table, the following statement would be used: ALTER TABLE MovieStar DROP birthdate.   These modifications allow you to

In [34]:
Markdown(answer.text)

Modifying a relation schema, or table definition, in SQL involves making changes to the existing schema, such as adding or deleting attributes.   To add an attribute, the ALTER TABLE statement can be used with the ADD keyword, followed by the attribute name and data type.   For example, to add a phone attribute to the MovieStar table as a fixed-length string of 16 bytes, the following statement would be used: ALTER TABLE MovieStar ADD phone CHAR(16).   Similarly, to delete an attribute, the ALTER TABLE statement can be used with the DROP keyword, followed by the attribute name.   For example, to delete the birthdate attribute from the MovieStar table, the following statement would be used: ALTER TABLE MovieStar DROP birthdate.   These modifications allow you to change the schema of a table without having to recreate it from scratch.

In [30]:

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
